In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Função para remover outliers
def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Caminhos para os arquivos de dados
file_paths = ['C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv']

# Processamento e previsão simples para cada arquivo
for file_path in file_paths:
    data = pd.read_csv(file_path)

    data['Data'] = pd.to_datetime(data['Data'])
    data = data.iloc[1:]

    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    data = remove_outliers(data, numeric_columns)

    # Dividindo os dados em treino e teste
    train_data = data[data['Data'] < pd.to_datetime("2020-01-01")]
    test_data = data[data['Data'] >= pd.to_datetime("2020-01-01")]

    # Preparando dados para previsão
    test_data['Previsto'] = test_data['Dif_Preco_Dolar'].shift(1)
    test_data = test_data.dropna()

    # Extraindo valores para avaliação e exibição
    datas = test_data['Data']
    y_test = test_data['Dif_Preco_Dolar'].values
    y_pred = test_data['Previsto'].values

    # Avaliando o modelo
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Exibindo resultados
    print(f"Resultados para {file_path}:")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R^2:", r2)
    print()

    # Filtrar para a data específica
    data_especifica = pd.to_datetime("2023-09-27")
    previsao_especifica = test_data[test_data['Data'] == data_especifica]

    # Verificando se há dados para essa data
    if not previsao_especifica.empty:
        valor_real = previsao_especifica['Dif_Preco_Dolar'].iloc[0]
        valor_previsto = previsao_especifica['Previsto'].iloc[0]
        print(f"Previsão para {data_especifica.date()}: Real: {valor_real}, Previsto: {valor_previsto}")
    else:
        print(f"Não há dados disponíveis para a data {data_especifica.date()}")


Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv:
MSE: 5.554405594405594
MAE: 1.8444755244755244
R^2: -0.9992394824013016

Previsão para 2023-09-27: Real: -1.16, Previsto: -1.09
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv:
MSE: 5.334765096952909
MAE: 1.8121883656509696
R^2: -0.9321274691314929

Previsão para 2023-09-27: Real: -1.16, Previsto: -1.09
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv:
MSE: 4.792730472103004
MAE: 1.7251502145922748
R^2: -0.8312852066574905

Previsão para 2023-09-27: Real: -1.16, Previsto: -1.09
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv:
MSE: 5.38777182320442
MAE: 1.8244751381215472
R^2: -0.9169725573471783

Previsão para 2023-09-27: Real: -1.16, Previsto: -1.09
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv:
MSE: 4.6574483050847455
MAE: 1.6984745762711864
R^2: -0.7957915431887048

Previsão para 2023-09-27: Real: -1.16, Pre

C:\Users\milen\AppData\Local\Temp\ipykernel_30592\325803400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Previsto'] = test_data['Dif_Preco_Dolar'].shift(1)
C:\Users\milen\AppData\Local\Temp\ipykernel_30592\325803400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Previsto'] = test_data['Dif_Preco_Dolar'].shift(1)
C:\Users\milen\AppData\Local\Temp\ipykernel_30592\325803400.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try